In [1]:
import os
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding as sym_padding
from time import time

In [2]:
def save_to_file(filename, data):
    with open(filename, 'wb') as f:
        f.write(data)

In [3]:
def load_from_file(filename):
    with open(filename, 'rb') as f:
        return f.read()

In [4]:
def generate_aes_key(length):
    key = os.urandom(length)
    save_to_file(f'aes_key_{length * 8}.key', key)
    return key

In [5]:
def aes_encrypt(data, key, mode):
    iv = os.urandom(16)
    if mode == 'ECB':
        cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    elif mode == 'CFB':
        cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padder = sym_padding.PKCS7(128).padder()
    padded_data = padder.update(data) + padder.finalize()
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()
    save_to_file('aes_encrypted.bin', iv + encrypted_data)

In [6]:
def aes_decrypt(key, mode):
    encrypted_data = load_from_file('aes_encrypted.bin')
    iv, encrypted_data = encrypted_data[:16], encrypted_data[16:]
    if mode == 'ECB':
        cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    elif mode == 'CFB':
        cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_data = decryptor.update(encrypted_data) + decryptor.finalize()
    unpadder = sym_padding.PKCS7(128).unpadder()
    data = unpadder.update(decrypted_data) + unpadder.finalize()
    print(data)

In [7]:
def generate_rsa_keys():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())
    public_key = private_key.public_key()
    save_to_file('rsa_private_key.pem', private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    ))
    save_to_file('rsa_public_key.pem', public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ))

In [8]:
def rsa_encrypt(data, public_key):
    encrypted_data = public_key.encrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    save_to_file('rsa_encrypted.bin', encrypted_data)

In [9]:
def rsa_decrypt(private_key):
    encrypted_data = load_from_file('rsa_encrypted.bin')
    decrypted_data = private_key.decrypt(
        encrypted_data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    print(decrypted_data)

In [10]:
def rsa_sign(data, private_key):
    signature = private_key.sign(
        data,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    save_to_file('rsa_signature.sig', signature)

In [11]:
def rsa_verify(data, signature, public_key):
    try:
        public_key.verify(
            signature,
            data,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("Verification successful")
    except Exception as e:
        print(f"Verification failed: {e}")

In [12]:
def sha256_hash(data):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(data)
    hash_value = digest.finalize()
    print(hash_value.hex())

In [13]:
def main():
    while True:
        print("Select operation:")
        print("1. AES Encryption")
        print("2. AES Decryption")
        print("3. RSA Encryption")
        print("4. RSA Decryption")
        print("5. RSA Signature")
        print("6. RSA Verification")
        print("7. SHA-256 Hashing")
        print("8. Exit")
        choice = int(input("Enter choice: "))
        if choice == 1:
            key_length = int(input("Enter AES key length (128 or 256): "))
            mode = input("Enter AES mode (ECB or CFB): ")
            key = generate_aes_key(key_length // 8)
            data = input("Enter data to encrypt: ").encode()
            start = time()
            aes_encrypt(data, key, mode)
            end = time()
            print(f"AES encryption took {end - start} seconds")
        elif choice == 2:
            key_length = int(input("Enter AES key length (128 or 256): "))
            mode = input("Enter AES mode (ECB or CFB): ")
            key = load_from_file(f'aes_key_{key_length}.key')
            start = time()
            aes_decrypt(key, mode)
            end = time()
            print(f"AES decryption took {end - start} seconds")
        elif choice == 3:
            generate_rsa_keys()
            data = input("Enter data to encrypt: ").encode()
            public_key = serialization.load_pem_public_key(load_from_file('rsa_public_key.pem'), backend=default_backend())
            start = time()
            rsa_encrypt(data, public_key)
            end = time()
            print(f"RSA encryption took {end - start} seconds")
        elif choice == 4:
            private_key = serialization.load_pem_private_key(load_from_file('rsa_private_key.pem'), password=None, backend=default_backend())
            start = time()
            rsa_decrypt(private_key)
            end = time()
            print(f"RSA decryption took {end - start} seconds")
        elif choice == 5:
            private_key = serialization.load_pem_private_key(load_from_file('rsa_private_key.pem'), password=None, backend=default_backend())
            data = input("Enter data to sign: ").encode()
            start = time()
            rsa_sign(data, private_key)
            end = time()
            print(f"RSA signature took {end - start} seconds")
        elif choice == 6:
            public_key = serialization.load_pem_public_key(load_from_file('rsa_public_key.pem'), backend=default_backend())
            data = input("Enter data to verify: ").encode()
            signature = load_from_file('rsa_signature.sig')
            start = time()
            rsa_verify(data, signature, public_key)
            end = time()
            print(f"RSA verification took {end - start} seconds")
        elif choice == 7:
            data = input("Enter data to hash: ").encode()
            start = time()
            sha256_hash(data)
            end = time()
            print(f"SHA-256 hashing took {end - start} seconds")
        elif choice == 8:
            break
        else:
            print("Invalid choice, please try again")

In [14]:
if __name__ == "__main__":
    main()

Select operation:
1. AES Encryption
2. AES Decryption
3. RSA Encryption
4. RSA Decryption
5. RSA Signature
6. RSA Verification
7. SHA-256 Hashing
8. Exit
AES encryption took 0.025913238525390625 seconds
Select operation:
1. AES Encryption
2. AES Decryption
3. RSA Encryption
4. RSA Decryption
5. RSA Signature
6. RSA Verification
7. SHA-256 Hashing
8. Exit
b'Deep Learning is fascinating!'
AES decryption took 0.0002543926239013672 seconds
Select operation:
1. AES Encryption
2. AES Decryption
3. RSA Encryption
4. RSA Decryption
5. RSA Signature
6. RSA Verification
7. SHA-256 Hashing
8. Exit
RSA encryption took 0.00038123130798339844 seconds
Select operation:
1. AES Encryption
2. AES Decryption
3. RSA Encryption
4. RSA Decryption
5. RSA Signature
6. RSA Verification
7. SHA-256 Hashing
8. Exit
b'RSA is a public key cryptography system.'
RSA decryption took 0.0011818408966064453 seconds
Select operation:
1. AES Encryption
2. AES Decryption
3. RSA Encryption
4. RSA Decryption
5. RSA Signature
